## 20230222
莹莹老师更新了gtf文件，经过测试发现，新的gtf文件中的Gene ID变成了纯数字的，读取的时候不是以字符串的形式，导致后面有一些问题，所以需要将新的gtf文件中的Gene ID改成字符串的形式，这样就可以正常读取了。

In [1]:
# 导入pandas库
import pandas as pd
import sys


In [2]:
ref = r"D:\githubku\my_jupyter\ref\merge1v1.txt"
input1 = r'D:\githubku\my_jupyter\20230223_check_new_gtf\NCBI_final_50.gene_abund.txt'

In [ ]:

input2 = sys.argv[2]
# output1 = sys.argv[3]
# output2 = sys.argv[4]
# # 输入
# input1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\SGI75460067.gene_abund.txt"
# input2 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\SGI75460067.gene_count.txt"
# path = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\"
# fileid = "SGI75460067"
# 输出
path=sys.argv[3]
fileid=sys.argv[4]

In [3]:
ref1_file = pd.read_csv(ref, sep='\t', engine='python')
Name_type = ref1_file.drop_duplicates(subset=['Gene Name'], keep='first')
# 以csv格式读入ref文件，tab为分隔符,列名分别设置为id，Gene Name，type，
ref1_file = pd.read_csv(ref, sep='\t', engine='python')
# 获取ref1_file的type列，去重,保存为dataframe类型,赋值给变量type_tpm,添加四列TPM>0,TPM>1,TPM>5,TPM>10
type_tpm = pd.DataFrame(ref1_file['Gene type'].unique())
type_tpm.columns = ['Gene type']
type_tpm['TPM>0'] = 0
type_tpm['TPM>1'] = 0
type_tpm['TPM>5'] = 0
type_tpm['TPM>10'] = 0

# 挑出NAME和type
# ref1_file中删除Gene Name列重复的行，保存为变量Name_type
Name_type = ref1_file.drop_duplicates(subset=['Gene Name'], keep='first')
# delete the column Gene ID,ENST
Name_type = Name_type.drop(['Gene ID', 'ENST'], axis=1)

# 挑出ENST和type

# 保留ref1_file中ENST和Gene type列，保存为变量ENST_type
ENST_type = ref1_file[['ENST', 'Gene type']]
# 删除ENST_type中ENST列为0的数据
ENST_type = ENST_type[ENST_type['ENST'] != '0']
# 删除ENST_type中ENST列重复的数据
ENST_type = ENST_type.drop_duplicates(['ENST'])

# 挑出GeneID和type
# 保留ref1_file中Gene ID和Gene type列，保存为变量ENST_type,删除ENST_type中ENST列重复的数据
Gene_ID_type = ref1_file[['Gene ID', 'Gene type']]
Gene_ID_type = Gene_ID_type.drop_duplicates(['Gene ID'])

# 挑出unassigned_type和type
# 获取ref1_file中Gene ID以unassigned开头的数据,保存为变量unassigned_type
unassigned_type = ref1_file[ref1_file['Gene ID'].str.contains(
    'unassigned_transcript_', na=False)]
# 获取ref1_file中gene name和gene id都存在的数据,保存为part3
part3 = ref1_file[ref1_file['Gene Name'].notnull() &
                  ref1_file['Gene ID'].notnull()]


In [11]:
## ***********PART1 START***********##

# 以csv格式读入input1文件，以tab为分隔符，不设置列名，保留第二列和最后一列数据
input1_file = pd.read_csv(input1, sep='\t', usecols=[
                          "Gene ID", "Gene Name", "TPM"], engine='python')
input1_file['Gene ID'] = input1_file['Gene ID'].astype(str)


In [12]:
# 将input1_file的Gene ID列中nan的数据替换为unknown
input1_file['Gene ID'] = input1_file['Gene ID'].fillna('unknown')

In [13]:
# 获取input1_file中Gene ID列只包含数字的数据
input1_number = input1_file[input1_file['Gene ID'].str.isdigit()]

In [14]:
# 获取input1_file的Gene ID列中ENST开头的数据
input1_ENST = input1_file[input1_file['Gene ID'].str.startswith('ENST')]
# 获取input1_file的Gene ID列中unknown开头的数据
input1_unknown = input1_file[input1_file['Gene ID'].str.startswith('unknown')]

In [17]:

# 获取input1_file中Gene ID列不只包含纯数字和不含ENST和unknown的数据
input1_other = input1_file[~input1_file['Gene ID'].str.isdigit()]
input1_other = input1_other[~input1_other['Gene ID'].str.startswith('ENST')]
input1_other = input1_other[~input1_other['Gene ID'].str.startswith('unknown')]

# 处理ENST开头的数据
# 以input1_ENST的Gene ID和ref1_file的ENST为键，input1_ENST为主表，合并input1_ENST和ref1_file，保存为变量ENST_tpm，并输出
ENST_tpm = pd.merge(input1_ENST, ENST_type,
                    left_on='Gene ID', right_on='ENST', how='left')
# 获取ENST_tpm中Gene type列为nan的数据m_nan
ENST_tpm_nan = ENST_tpm[ENST_tpm['Gene type'].isnull()]
# ENST_tpm_nan保留Gene ID、Gene Name和TPM列
ENST_tpm_nan = ENST_tpm_nan[['Gene ID', 'Gene Name', 'TPM']]
# 删除ENST_tpm中Gene type列为nan的数据
ENST_tpm = ENST_tpm[ENST_tpm['Gene type'].notnull()]

ENST_tpm_df = type_tpm
# 遍历ENST_tpm_df索引，统计type_tpmTPM>0,TPM>1,TPM>5,TPM>10的数量
for i in ENST_tpm_df.index:
    ENST_tpm_type = ENST_tpm[ENST_tpm['Gene type']
                             == ENST_tpm_df.loc[i, 'Gene type']]
    for j in ENST_tpm_type.index:
        if ENST_tpm_type.loc[j, 'TPM'] > 0:
            ENST_tpm_df.loc[i, 'TPM>0'] += 1
        if ENST_tpm_type.loc[j, 'TPM'] > 1:
            ENST_tpm_df.loc[i, 'TPM>1'] += 1
        if ENST_tpm_type.loc[j, 'TPM'] > 5:
            ENST_tpm_df.loc[i, 'TPM>5'] += 1
        if ENST_tpm_type.loc[j, 'TPM'] > 10:
            ENST_tpm_df.loc[i, 'TPM>10'] += 1

# 以input1_number的Gene ID和ref1_file的Gene ID为键，input1_number为主表，合并input1_number和ref1_file，保存为变量Gene_ID_tpm，并输出
Gene_ID_tpm = pd.merge(input1_number, Gene_ID_type,
                       left_on='Gene ID', right_on='Gene ID', how='left')

# 获得Gene_ID_tpm中Gene type为nan的数据，保存为变量Gene_ID_tpm_nan
Gene_ID_tpm_nan = Gene_ID_tpm[Gene_ID_tpm['Gene type'].isnull()]
# 保留Gene_ID_tpm_nan的Gene ID、Gene Name和TPM列
Gene_ID_tpm_nan = Gene_ID_tpm_nan[['Gene ID', 'Gene Name', 'TPM']]
# 删除Gene_ID_tpm中Gene type为nan的数据
Gene_ID_tpm = Gene_ID_tpm[Gene_ID_tpm['Gene type'].notnull()]

# 只能通过gene name来进行比对的数据，还有一部分不能通过gene name查到
# 合并Gene_ID_tpm_nan、ENST_tpm_nan、input1_other、input1_unknown
input1_byname = pd.concat(
    [Gene_ID_tpm_nan, ENST_tpm_nan, input1_other, input1_unknown])
# 以input1_byname的Gene Name和ref1_file的Gene Name为键，input1_byname为主表
Name_tpm = pd.merge(input1_byname, Name_type,
                    left_on='Gene Name', right_on='Gene Name', how='left')
# 遍历NAME_tpm_df索引，统计NAME_tpm_dfTPM>0,TPM>1,TPM>5,TPM>10的数量
NAME_tpm_df = type_tpm
for i in NAME_tpm_df.index:
    # 获取Name_tpm中Gene type为NAME_tpm_df中Gene type的数据
    Name_tpm_type = Name_tpm[Name_tpm['Gene type']
                             == NAME_tpm_df.loc[i, 'Gene type']]
    # 遍历Name_tpm_type索引，统计TPM>0,TPM>1,TPM>5,TPM>10的数量
    for j in Name_tpm_type.index:
        if Name_tpm_type.loc[j, 'TPM'] > 0:
            NAME_tpm_df.loc[i, 'TPM>0'] += 1
        if Name_tpm_type.loc[j, 'TPM'] > 1:
            NAME_tpm_df.loc[i, 'TPM>1'] += 1
        if Name_tpm_type.loc[j, 'TPM'] > 5:
            NAME_tpm_df.loc[i, 'TPM>5'] += 1
        if Name_tpm_type.loc[j, 'TPM'] > 10:
            NAME_tpm_df.loc[i, 'TPM>10'] += 1

# 遍历ID_tpm_df索引，统计ID_tpm_dfTPM>0,TPM>1,TPM>5,TPM>10的数量
ID_tpm_df = type_tpm
for i in ID_tpm_df.index:
    ID_tpm_type = Gene_ID_tpm[Gene_ID_tpm['Gene type']
                              == ID_tpm_df.loc[i, 'Gene type']]
    for j in ID_tpm_type.index:
        if ID_tpm_type.loc[j, 'TPM'] > 0:
            ID_tpm_df.loc[i, 'TPM>0'] += 1
        if ID_tpm_type.loc[j, 'TPM'] > 1:
            ID_tpm_df.loc[i, 'TPM>1'] += 1
        if ID_tpm_type.loc[j, 'TPM'] > 5:
            ID_tpm_df.loc[i, 'TPM>5'] += 1
        if ID_tpm_type.loc[j, 'TPM'] > 10:
            ID_tpm_df.loc[i, 'TPM>10'] += 1


In [18]:
type_tpm

,Gene type,TPM>0,TPM>1,TPM>5,TPM>10
0,lncRNA,462,406,348,332
1,miRNA,98,98,98,98
2,snoRNA,8,8,8,8
3,other,6,0,0,0
4,protein_coding,690,560,486,462
5,pseudo,816,798,780,774
6,rRNA,0,0,0,0
7,Mt_rRNA,0,0,0,0
8,scRNA,0,0,0,0
9,snRNA,0,0,0,0


In [ ]:

# type_tpm.to_csv(output1, index=False)
# print('finished'+str(output1))

type_tpm.to_csv(path+'/'+fileid+".TPM_count.txt", index=False)
print('finished'+str(path+'/'+fileid+".TPM_count.txt"))